In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import pandas as pd
import warnings
import sys
import numpy as np
import scipy as sc
import random
from scipy import stats
from numpy import log10, sqrt

mydir = '/Users/kenlocey/GitHub/HACRP-HAIs/'
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def def_display_df_misses(df):
    df = df[df['Payment Reduction Reproduced?'] == 0]
    items = ['file_year', 'HAI Measures End Date', 'HAI Measures Start Date',
             'CAUTI Footnote', 'CAUTI W Z Score', 'CAUTI SIR W Z Score',
             'CDI Footnote', 'CDI W Z Score', 'CDI SIR W Z Score',
             'CLABSI Footnote', 'CLABSI W Z Score', 'CLABSI SIR W Z Score',
             'MRSA Footnote', 'MRSA W Z Score', 'MRSA SIR W Z Score',
             'SSI Footnote', 'SSI W Z Score', 'SSI SIR W Z Score',
             'PSI-90 Footnote', 'PSI-90 W Z Score', 'PSI-90 SIR W Z Score',
             'Total HAC Footnote', 'Total HAC Score', 'Total HAC Score (derived)',
             'Payment Reduction Footnote', 'Payment Reduction', 
             'Payment Reduction (derived)', 'Payment Reduction Reproduced?',
             ]
    return df.filter(items=items)


def Winsorize_it(x, WinZs):
    
    x2 = []
    for i, val in enumerate(x):
        wz = float(WinZs[i])
        val = float(val)
        if np.isnan(wz) == True:
            x2.append(np.nan)
        else:
            x2.append(val)
    
    p5 = np.nanpercentile(x2, 5)
    p95 = np.nanpercentile(x2, 95)
    WinScores = []
    
    for i, val in enumerate(x2):
        wz = float(WinZs[i])
        val = float(val)
        
        if np.isnan(wz) == True:
            WinScores.append(np.nan)
            
        elif np.isnan(wz) == False:
            if val >= p5 and val <= p95:
                WinScores.append(val)
            elif val < p5:
                WinScores.append(p5)
            elif val > p95:
                WinScores.append(p95)
            elif np.isnan(val) == True:
                #print('val:', val, '|', WinZs[i])
                WinScores.append(np.nan)
        
    
    return WinScores
        

def ZScore_it(x, WinZs):
    
    x2 = []
    for i, val in enumerate(x):
        wz = float(WinZs[i])
        val = float(val)
        if np.isnan(wz) == True:
            x2.append(np.nan)
        else:
            x2.append(val)
    
    x2 = np.array(x2)
    avg = np.nanmean(x2)
    std = np.nanstd(x2)
    zscores = (x2 - avg) / std
    return zscores


## Load HAC file

In [2]:
hac_df = pd.read_pickle(mydir + "data/CareCompare_data/CombinedFiles_HACRP/Facility.pkl")
hac_df = hac_df[hac_df['file_year'] == '2020']
hac_df = hac_df[hac_df['file_month'] == '10']

print('HACRP years:', sorted(hac_df['Fiscal Year'].unique()))

features = ['CAUTI Score', 'CLABSI Score', 'CDI Score', 'MRSA Score', 'SSI Score',
            'Total HAC Score', 'Domain 1 Score', 'AHRQ PSI-90 Score', 'Domain 2 Score',
            'CAUTI W Z Score', 'CLABSI W Z Score', 'MRSA W Z Score', 'CDI W Z Score', 
            'SSI W Z Score', 'PSI-90 W Z Score']
for f in features:
    hac_df[f] = hac_df[f].astype(str)
    hac_df[f] = hac_df[f].str.replace('*', '')
    hac_df[f] = pd.to_numeric(hac_df[f], errors='coerce')

hac_df.dropna(how='all', axis=1, inplace=True)
hac_df.head()

HACRP years: [2020]


,CAUTI Footnote,CAUTI W Z Score,CDI Footnote,CDI W Z Score,CLABSI Footnote,CLABSI W Z Score,Facility ID,Facility Name,Fiscal Year,HAI Measures End Date,HAI Measures Start Date,MRSA Footnote,MRSA W Z Score,PSI-90 End Date,PSI-90 Footnote,PSI-90 Start Date,PSI-90 W Z Score,Payment Reduction,Payment Reduction Footnote,SSI Footnote,SSI W Z Score,State,Total HAC Footnote,Total HAC Score,file_month,file_year
0,NaN,0.1160,NaN,-0.4571,NaN,1.0325,010001,SOUTHEAST ALABAMA MEDICAL CENTER,2020,12/31/2018,01/01/2017,NaN,-0.5533,06/30/2018,NaN,07/01/2016,0.0144,No,NaN,NaN,-1.2403,AL,NaN,-0.1813,10,2020
1,NaN,-0.1406,NaN,-0.6434,NaN,2.1941,010005,MARSHALL MEDICAL CENTERS SOUTH CAMPUS,2020,12/31/2018,01/01/2017,NaN,2.2502,06/30/2018,NaN,07/01/2016,0.1869,Yes,NaN,NaN,-0.1681,AL,NaN,0.6132,10,2020
2,NaN,0.7792,NaN,-0.7786,NaN,0.3263,010006,NORTH ALABAMA MEDICAL CENTER,2020,12/31/2018,01/01/2017,NaN,0.8325,06/30/2018,NaN,07/01/2016,0.2308,No,NaN,NaN,-0.6508,AL,NaN,0.1232,10,2020
3,NaN,-1.5354,NaN,-0.7877,5.0,NaN,010007,MIZELL MEMORIAL HOSPITAL,2020,12/31/2018,01/01/2017,5.0,NaN,06/30/2018,NaN,07/01/2016,-0.0548,No,NaN,5.0,NaN,AL,NaN,-0.7926,10,2020
4,5.0,NaN,5.0,NaN,5.0,NaN,010008,CRENSHAW COMMUNITY HOSPITAL,2020,12/31/2018,01/01/2017,5.0,NaN,06/30/2018,NaN,07/01/2016,-0.0047,No,NaN,5.0,NaN,AL,NaN,-0.0047,10,2020


## Format dates within the HAC file 

In [3]:

## Convert all other date data to default datetime

hac_df['HAI Measures Start Date'] = pd.to_datetime(hac_df['HAI Measures Start Date'])
hac_df.style.format({'HAI Measures Start Date': lambda t: t.strftime("%Y-%m-%d")})

hac_df['HAI Measures End Date'] = pd.to_datetime(hac_df['HAI Measures End Date'])
hac_df.style.format({'HAI Measures End Date': lambda t: t.strftime("%Y-%m-%d")})

hac_df = pd.concat([hac_df], axis=0)
hac_df.sort_values(by='HAI Measures End Date', inplace=True)

hac_df.rename(columns={'HAI Measures Start Date': 'Start Date', 'HAI Measures End Date': 'End Date'}, inplace=True)


## Load and merge HAI files

In [4]:
##############################   CAUTI   ################################################

cauti_df = pd.read_pickle(mydir + "1_preprocess_CareCompare_data/preprocessed_HAI_data/CAUTI_Data.pkl")
cauti_df = cauti_df.filter(items=['Facility ID', 'CAUTI Urinary Catheter Days (ICUs + select wards)', 
                                  'CAUTI Observed Cases (ICUs + select wards)', 
                                  'CAUTI Predicted Cases (ICUs + select wards)', 
                                  'Start Date', 'End Date'], axis=1)

cauti_df.rename(columns={'CAUTI Urinary Catheter Days (ICUs + select wards)': 'CAUTI Urinary Catheter Days', 
                         'CAUTI Observed Cases (ICUs + select wards)': 'CAUTI Observed Cases', 
                         'CAUTI Predicted Cases (ICUs + select wards)': 'CAUTI Predicted Cases', 
                        }, inplace=True)

features = ['CAUTI Urinary Catheter Days', 'CAUTI Observed Cases', 'CAUTI Predicted Cases']
for f in features:
    cauti_df[f] = cauti_df[f].astype(str)
    cauti_df[f] = pd.to_numeric(cauti_df[f], errors='coerce')

cauti_df['Start Date'] = pd.to_datetime(cauti_df['Start Date'])
cauti_df.style.format({'Start Date': lambda t: t.strftime("%Y-%m-%d")})
cauti_df['End Date'] = pd.to_datetime(cauti_df['End Date'])
cauti_df.style.format({'End Date': lambda t: t.strftime("%Y-%m-%d")})
cauti_df = cauti_df[(cauti_df['Start Date'].isin(hac_df['Start Date'].unique())) | (cauti_df['End Date'].isin(hac_df['End Date'].unique()))]

cauti_df.head()


,Facility ID,CAUTI Urinary Catheter Days,CAUTI Observed Cases,CAUTI Predicted Cases,Start Date,End Date
48,670102,NaN,NaN,NaN,2017-01-01,2017-12-31
49,190297,322.0,0.0,0.158,2018-01-01,2018-12-31
75,061327,477.0,0.0,0.312,2018-01-01,2018-12-31
87,150002,12860.0,10.0,18.756,2018-01-01,2018-12-31
90,230301,8.0,0.0,0.004,2017-01-01,2017-12-31


In [5]:
##############################   CLABSI   ###############################################

clabsi_df = pd.read_pickle(mydir + "1_preprocess_CareCompare_data/preprocessed_HAI_data/CLABSI_Data.pkl")
clabsi_df = clabsi_df.filter(items=['Facility ID', 'CLABSI Device Days (ICUs + select wards)', 
                                    'CLABSI Observed Cases (ICUs + select wards)', 
                                    'CLABSI Predicted Cases (ICUs + select wards)', 
                                    'Start Date', 'End Date'], axis=1)

clabsi_df.rename(columns={'CLABSI Device Days (ICUs + select wards)': 'CLABSI Device Days', 
                          'CLABSI Observed Cases (ICUs + select wards)': 'CLABSI Observed Cases', 
                          'CLABSI Predicted Cases (ICUs + select wards)': 'CLABSI Predicted Cases', 
                        }, inplace=True)


features = ['CLABSI Device Days', 'CLABSI Observed Cases', 'CLABSI Predicted Cases']
for f in features:
    clabsi_df[f] = clabsi_df[f].astype(str)
    clabsi_df[f] = pd.to_numeric(clabsi_df[f], errors='coerce')

clabsi_df['Start Date'] = pd.to_datetime(clabsi_df['Start Date'])
clabsi_df.style.format({'Start Date': lambda t: t.strftime("%Y-%m-%d")})
clabsi_df['End Date'] = pd.to_datetime(clabsi_df['End Date'])
clabsi_df.style.format({'End Date': lambda t: t.strftime("%Y-%m-%d")})
clabsi_df = clabsi_df[(clabsi_df['Start Date'].isin(hac_df['Start Date'].unique())) | (clabsi_df['End Date'].isin(hac_df['End Date'].unique()))]

clabsi_df.head()

,Facility ID,CLABSI Device Days,CLABSI Observed Cases,CLABSI Predicted Cases,Start Date,End Date
1,390008,297.0,0.0,0.174,2017-01-01,2017-12-31
72,100288,12198.0,13.0,13.926,2018-01-01,2018-12-31
78,310130,1041.0,0.0,0.733,2017-01-01,2017-12-31
85,170049,6998.0,1.0,5.587,2017-01-01,2017-12-31
93,181311,NaN,NaN,NaN,2017-01-01,2017-12-31


In [6]:

##############################   MRSA   ###############################################

mrsa_df = pd.read_pickle(mydir + "1_preprocess_CareCompare_data/preprocessed_HAI_data/MRSA_Data.pkl")
mrsa_df = mrsa_df.filter(items=['Facility ID', 'MRSA patient days', 
                                'MRSA Observed Cases', 'MRSA Predicted Cases', 
                                'Start Date', 'End Date'], axis=1)
mrsa_df['Start Date'] = pd.to_datetime(mrsa_df['Start Date'])
mrsa_df.style.format({'Start Date': lambda t: t.strftime("%Y-%m-%d")})
mrsa_df['End Date'] = pd.to_datetime(mrsa_df['End Date'])
mrsa_df.style.format({'End Date': lambda t: t.strftime("%Y-%m-%d")})

mrsa_df = mrsa_df[(mrsa_df['Start Date'].isin(hac_df['Start Date'].unique())) | (mrsa_df['End Date'].isin(hac_df['End Date'].unique()))]

##############################   CDI   ###############################################

cdi_df = pd.read_pickle(mydir + "1_preprocess_CareCompare_data/preprocessed_HAI_data/CDI_Data.pkl")
cdi_df = cdi_df.filter(items=['Facility ID', 'CDIFF patient days', 
                              'CDIFF Observed Cases', 'CDIFF Predicted Cases', 
                              'Start Date', 'End Date'], axis=1)
cdi_df['Start Date'] = pd.to_datetime(cdi_df['Start Date'])
cdi_df.style.format({'Start Date': lambda t: t.strftime("%Y-%m-%d")})
cdi_df['End Date'] = pd.to_datetime(cdi_df['End Date'])
cdi_df.style.format({'End Date': lambda t: t.strftime("%Y-%m-%d")})

cdi_df = cdi_df[(cdi_df['Start Date'].isin(hac_df['Start Date'].unique())) | (cdi_df['End Date'].isin(hac_df['End Date'].unique()))]

###################  Merge CAUTI, CLABSI, MRSA, and CDI  ####################################

hai_df = cauti_df.merge(clabsi_df, on=['Facility ID', 'Start Date', 'End Date'], how='outer')
hai_df = hai_df.merge(mrsa_df, on=['Facility ID', 'Start Date', 'End Date'], how='outer')
hai_df = hai_df.merge(cdi_df, on=['Facility ID', 'Start Date', 'End Date'], how='outer')


####################  Drop duplicate rows resulting from merger #############################

hai_df.drop_duplicates(inplace=True)


########################  Conversions to numeric  #############################################

features = ['CAUTI Urinary Catheter Days', 'CLABSI Device Days',
            'MRSA patient days', 'CDIFF patient days',
            'CLABSI Observed Cases', 'CLABSI Predicted Cases', 
            'MRSA Observed Cases', 'MRSA Predicted Cases', 
            'CDIFF Observed Cases', 'CDIFF Predicted Cases', 
            'CAUTI Observed Cases', 'CAUTI Predicted Cases', 
           ]

for f in features:
    hai_df[f] = hai_df[f].astype(str)
    hai_df[f] = hai_df[f].str.replace('*', '')
    hai_df[f] = hai_df[f].str.replace(' ', '')
    hai_df[f] = pd.to_numeric(hai_df[f], errors='coerce')
    
hai_df['Total device days'] = hai_df['CLABSI Device Days'] + hai_df['CAUTI Urinary Catheter Days']


########################  Reorder columns  #############################################

col_to_move = hai_df.pop('CAUTI Urinary Catheter Days')
hai_df.insert(hai_df.shape[1] - 2, 'CAUTI Urinary Catheter Days', col_to_move)

print(hai_df.shape)
hai_df.head()

(9550, 16)


,Facility ID,CAUTI Observed Cases,CAUTI Predicted Cases,Start Date,End Date,CLABSI Device Days,CLABSI Observed Cases,CLABSI Predicted Cases,MRSA patient days,MRSA Observed Cases,MRSA Predicted Cases,CDIFF patient days,CDIFF Observed Cases,CAUTI Urinary Catheter Days,CDIFF Predicted Cases,Total device days
0,670102,NaN,NaN,2017-01-01,2017-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,190297,0.0,0.158,2018-01-01,2018-12-31,5.0,0.0,0.003,1016.0,0.0,0.018,1016.0,0.0,322.0,0.242,327.0
2,061327,0.0,0.312,2018-01-01,2018-12-31,110.0,0.0,0.030,3964.0,0.0,0.083,3705.0,0.0,477.0,1.278,587.0
3,150002,10.0,18.756,2018-01-01,2018-12-31,14631.0,11.0,15.044,85670.0,4.0,7.102,80090.0,32.0,12860.0,66.269,27491.0
4,230301,0.0,0.004,2017-01-01,2017-12-31,NaN,NaN,NaN,1483.0,0.0,0.028,1483.0,0.0,8.0,0.323,NaN


## Filter HAI data on start dates and end dates that match those in the HAC file.

In [7]:
#########  Filter on start dates and end dates that match those in the HAC file  #######

hai_df = hai_df[(hai_df['Start Date'].isin(['2017-01-01', '2018-01-01'])) & (hai_df['End Date'].isin(['2017-12-31', '2018-12-31']))]

print(hai_df.shape)
print(hai_df['Start Date'].unique())
print(hai_df['End Date'].unique())

(9550, 16)
['2017-01-01T00:00:00.000000000' '2018-01-01T00:00:00.000000000']
['2017-12-31T00:00:00.000000000' '2018-12-31T00:00:00.000000000']


## Drop hospitals from HAI data that are not contained in HAC data.

In [8]:
# drop hospitals in HAI data that are not in HAC data
hai_df = hai_df[hai_df['Facility ID'].isin(hac_df['Facility ID'].unique())]

In [9]:
# hospitals in the hac data but not in the hai data ...
#tdf = hac_df.copy(deep=True) 
tdf = hac_df[~hac_df['Facility ID'].isin(hai_df['Facility ID'].unique())]
hac_df = hac_df[hac_df['Facility ID'].isin(hai_df['Facility ID'].unique())]

print(len(tdf['Facility ID'].unique()), 'hospitals in HAC dataset but not in HAI dataset\n')


tdf.head()

6 hospitals in HAC dataset but not in HAI dataset



,CAUTI Footnote,CAUTI W Z Score,CDI Footnote,CDI W Z Score,CLABSI Footnote,CLABSI W Z Score,Facility ID,Facility Name,Fiscal Year,End Date,Start Date,MRSA Footnote,MRSA W Z Score,PSI-90 End Date,PSI-90 Footnote,PSI-90 Start Date,PSI-90 W Z Score,Payment Reduction,Payment Reduction Footnote,SSI Footnote,SSI W Z Score,State,Total HAC Footnote,Total HAC Score,file_month,file_year
2316,5.0,NaN,5.0,NaN,5.0,NaN,370240,INTEGRIS COMMUNITY HOSPITAL - COUNCIL CROSSING,2020,2018-12-31,2017-01-01,5.0,NaN,06/30/2018,5.0,07/01/2016,NaN,No,NaN,5.0,NaN,OK,5.0,NaN,10,2020
3149,5.0,NaN,5.0,NaN,5.0,NaN,520210,MARSHFIELD MEDICAL CENTER - EAU CLAIRE,2020,2018-12-31,2017-01-01,5.0,NaN,06/30/2018,5.0,07/01/2016,NaN,No,NaN,5.0,NaN,WI,5.0,NaN,10,2020
490,5.0,NaN,5.0,NaN,5.0,NaN,050784,ADVENTIST HEALTH TULARE,2020,2018-12-31,2017-01-01,5.0,NaN,06/30/2018,5.0,07/01/2016,NaN,No,NaN,5.0,NaN,CA,5.0,NaN,10,2020
195,5.0,NaN,5.0,NaN,5.0,NaN,040156,"BAPTIST MEMORIAL HOSPITAL-CRITTENDEN, INC",2020,2018-12-31,2017-01-01,5.0,NaN,06/30/2018,5.0,07/01/2016,NaN,No,NaN,5.0,NaN,AR,5.0,NaN,10,2020
872,5.0,NaN,5.0,NaN,5.0,NaN,130073,GROVE CREEK MEDICAL CENTER,2020,2018-12-31,2017-01-01,5.0,NaN,06/30/2018,5.0,07/01/2016,NaN,No,NaN,5.0,NaN,ID,5.0,NaN,10,2020


## Correct HAI file for non-duplicate rows having duplicate dates

**Problem:** Some rows for the same provider have duplicate measurement dates but different values for observed cases, predicted cases, etc. This results from each year having multiple (quarterly) files, the data within which can vary among files. Additionally, the most recent file for each year is not always the right file to use. 

**Need:** Since only one row can be used, we need to figure out which row should be used.

**Solution:** Select the row with the greatest totals for predicted cases for each HAI.

In [10]:
##############  Label rows that have duplicate dates (per provider) ####################
##############  For each provider with rows having duplicate dates,  ###################
###########  keep the last row (will have greatest number of total device days)  #######

hai_df['duplicated dates'] = hai_df.duplicated(subset=['Facility ID', 'Start Date', 'End Date'], keep=False)

hai_df.sort_values(by=['Facility ID', 'Start Date', 'End Date', 
                       'MRSA Predicted Cases', 'CAUTI Predicted Cases',  
                       'MRSA Observed Cases', 'CAUTI Observed Cases', 
                       'CDIFF Predicted Cases', 'CLABSI Predicted Cases',
                       'CDIFF Observed Cases', 'CLABSI Observed Cases',
                       ], inplace=True, ascending=False)

hai_df.drop_duplicates(subset=['Facility ID', 'Start Date', 'End Date'], inplace=True, keep='first')

print(hai_df.shape)
hai_df.head()

(6422, 17)


,Facility ID,CAUTI Observed Cases,CAUTI Predicted Cases,Start Date,End Date,CLABSI Device Days,CLABSI Observed Cases,CLABSI Predicted Cases,MRSA patient days,MRSA Observed Cases,MRSA Predicted Cases,CDIFF patient days,CDIFF Observed Cases,CAUTI Urinary Catheter Days,CDIFF Predicted Cases,Total device days,duplicated dates
8932,670128,NaN,NaN,2018-01-01,2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1197,670125,NaN,NaN,2018-01-01,2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2096,670124,NaN,NaN,2018-01-01,2018-12-31,NaN,NaN,NaN,500.0,0.0,0.009,500.0,0.0,NaN,0.117,NaN,False
4712,670124,0.0,0.003,2017-01-01,2017-12-31,NaN,NaN,NaN,62.0,0.0,0.001,62.0,0.0,5.0,0.011,NaN,False
4992,670122,1.0,2.355,2018-01-01,2018-12-31,2913.0,0.0,2.19,29355.0,0.0,1.042,27541.0,8.0,3466.0,17.869,6379.0,False


## Aggregate annual HAI data into biennial data

Purpose: Match the biennial measurement periods of HAC data


In [11]:
start_dates = hac_df['Start Date'].tolist()
end_dates = hac_df['End Date'].tolist()
prvdrs = hac_df['Facility ID'].tolist()

total_device_days = []

cauti_days = []
clabsi_days = []
mrsa_days = []
cdi_days = []

cauti_pred = []
clabsi_pred = []
mrsa_pred = []
cdi_pred = []

cauti_obs = []
clabsi_obs = []
mrsa_obs = []
cdi_obs = []

for i, start in enumerate(start_dates):
    end = end_dates[i]
    prvdr = prvdrs[i]
    
    tdf = hai_df[hai_df['Facility ID'] == prvdr]
    tdf = tdf[(tdf['Start Date'] == start) | (tdf['End Date'] == end)]
    
    if tdf.shape[0] == 1:
        pass
        #print('tdf.shape[0]:', 1)
        #print('hospital:', prvdr)
        #print(tdf['Start Date'].unique())
        #print(tdf['End Date'].unique())
        
    if tdf.shape[0] > 2:
        print('tdf.shape[0] = ', tdf.shape[0])
        for date_ in ['Start Date', 'End Date']:
            tdf['duplicated dates'] = tdf.duplicated(subset=[date_], keep=False)
            tdf.sort_values(by=[
                                'CAUTI Observed Cases', 'CLABSI Observed Cases',
                                'MRSA Observed Cases', 'CDIFF Observed Cases',
                                ], inplace=True, ascending=True)

            tdf.drop_duplicates(subset=[date_], inplace=True, keep='last')

        if tdf.shape[0] > 2:
            print('Error:')
            print("tdf.shape[0] > 2:", tdf.shape[0])
            print(start)
            print(end)
            print(tdf.head())
            break
        
    total_device_days.append(np.nansum(tdf['Total device days']))
    cauti_days.append(np.nansum(tdf['CAUTI Urinary Catheter Days']))
    clabsi_days.append(np.nansum(tdf['CLABSI Device Days']))
    mrsa_days.append(np.nansum(tdf['MRSA patient days']))
    cdi_days.append(np.nansum(tdf['CDIFF patient days']))
        
    cauti_pred.append(np.nansum(tdf['CAUTI Predicted Cases']))
    clabsi_pred.append(np.nansum(tdf['CLABSI Predicted Cases']))
    mrsa_pred.append(np.nansum(tdf['MRSA Predicted Cases']))
    cdi_pred.append(np.nansum(tdf['CDIFF Predicted Cases']))

    cauti_obs.append(np.nansum(tdf['CAUTI Observed Cases']))
    clabsi_obs.append(np.nansum(tdf['CLABSI Observed Cases']))
    mrsa_obs.append(np.nansum(tdf['MRSA Observed Cases']))
    cdi_obs.append(np.nansum(tdf['CDIFF Observed Cases']))
    

## Add HAI data to the HAC dataframe and save

In [12]:
hac_df['Total device days'] = total_device_days
hac_df['CAUTI Urinary Catheter Days'] = cauti_days
hac_df['CLABSI Device Days'] = clabsi_days
hac_df['MRSA patient days'] = mrsa_days
hac_df['CDI patient days'] = cdi_days

hac_df['CAUTI Observed Cases'] = cauti_obs
hac_df['CLABSI Observed Cases'] = clabsi_obs
hac_df['MRSA Observed Cases'] = mrsa_obs
hac_df['CDI Observed Cases'] = cdi_obs

hac_df['CAUTI Predicted Cases'] = cauti_pred
hac_df['CLABSI Predicted Cases'] = clabsi_pred
hac_df['MRSA Predicted Cases'] = mrsa_pred
hac_df['CDI Predicted Cases'] = cdi_pred

hac_df['CAUTI derived SIR'] = np.round(hac_df['CAUTI Observed Cases'] / hac_df['CAUTI Predicted Cases'],4)
hac_df['CLABSI derived SIR'] = np.round(hac_df['CLABSI Observed Cases'] / hac_df['CLABSI Predicted Cases'],4)
hac_df['MRSA derived SIR'] = np.round(hac_df['MRSA Observed Cases'] / hac_df['MRSA Predicted Cases'],4)
hac_df['CDI derived SIR'] = np.round(hac_df['CDI Observed Cases'] / hac_df['CDI Predicted Cases'],4)

print('hac_df.shape:', hac_df.shape)
print(len(hac_df['Facility ID'].unique()), 'hospitals in 2020 HACRP')
#hac_df.to_pickle('~/GitHub/HACRP-HAIs/data/merged_HAC_HAI/merged_HAI_HAC_2020.pkl', protocol=5)
hac_df.head()



hac_df.shape: (3218, 43)
3218 hospitals in 2020 HACRP


,CAUTI Footnote,CAUTI W Z Score,CDI Footnote,CDI W Z Score,CLABSI Footnote,CLABSI W Z Score,Facility ID,Facility Name,Fiscal Year,End Date,Start Date,MRSA Footnote,MRSA W Z Score,PSI-90 End Date,PSI-90 Footnote,PSI-90 Start Date,PSI-90 W Z Score,Payment Reduction,Payment Reduction Footnote,SSI Footnote,SSI W Z Score,State,Total HAC Footnote,Total HAC Score,file_month,file_year,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,MRSA patient days,CDI patient days,CAUTI Observed Cases,CLABSI Observed Cases,MRSA Observed Cases,CDI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,MRSA Predicted Cases,CDI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,MRSA derived SIR,CDI derived SIR
0,NaN,0.1160,NaN,-0.4571,NaN,1.0325,010001,SOUTHEAST ALABAMA MEDICAL CENTER,2020,2018-12-31,2017-01-01,NaN,-0.5533,06/30/2018,NaN,07/01/2016,0.0144,No,NaN,NaN,-1.2403,AL,NaN,-0.1813,10,2020,41948.0,26809.0,15139.0,196023.0,194737.0,26.0,16.0,6.0,79.0,29.932,12.845,11.913,141.068,0.8686,1.2456,0.5037,0.5600
2143,NaN,1.1915,NaN,0.3543,NaN,-0.1729,360048,UNIVERSITY OF TOLEDO MEDICAL CENTER,2020,2018-12-31,2017-01-01,NaN,0.6308,06/30/2018,NaN,07/01/2016,2.3575,Yes,NaN,NaN,0.2022,OH,NaN,0.7605,10,2020,24770.0,12345.0,12425.0,99785.0,99785.0,22.0,8.0,9.0,58.0,15.327,12.497,7.671,69.863,1.4354,0.6402,1.1732,0.8302
2144,NaN,-0.4750,NaN,-0.0424,NaN,-0.8572,360051,MIAMI VALLEY HOSPITAL,2020,2018-12-31,2017-01-01,NaN,0.8131,06/30/2018,NaN,07/01/2016,-1.3464,No,NaN,NaN,0.0824,OH,NaN,-0.3042,10,2020,83234.0,36627.0,46607.0,382409.0,338818.0,32.0,15.0,38.0,180.0,57.329,50.757,29.775,257.787,0.5582,0.2955,1.2762,0.6983
2145,NaN,-0.6099,NaN,1.0184,NaN,0.8097,360054,HOLZER MEDICAL CENTER,2020,2018-12-31,2017-01-01,NaN,0.9299,06/30/2018,NaN,07/01/2016,1.1226,Yes,NaN,NaN,0.6038,OH,NaN,0.6457,10,2020,5997.0,4079.0,1918.0,39461.0,36711.0,2.0,2.0,3.0,24.0,4.109,1.763,2.236,22.834,0.4867,1.1344,1.3417,1.0511
2146,4.0,NaN,4.0,NaN,4.0,NaN,360055,TRUMBULL REGIONAL MEDICAL CENTER,2020,2018-12-31,2017-01-01,4.0,NaN,06/30/2018,4.0,07/01/2016,NaN,No,NaN,4.0,NaN,OH,4.0,NaN,10,2020,23530.0,13609.0,9921.0,77415.0,77373.0,13.0,11.0,3.0,48.0,18.768,10.443,5.210,57.329,0.6927,1.0533,0.5758,0.8373


## Generate Winsorized z-scores

In [13]:
hais = ['CAUTI', 'CLABSI', 'MRSA', 'CDI']

df_yr = hac_df.copy(deep=True)
    
for i, hai in enumerate(hais):
    tdf2 = df_yr[~df_yr[hai + ' Footnote'].isin([18, '18', '18 ', ' 18', 
                                              5, '5', ' 5', '5 ',
                                              4, '4', ' 4', '4 ',
                                              ])]
      
    reported_winZ = tdf2[hai + ' W Z Score'].tolist()
    sirs = tdf2[hai + ' derived SIR'].tolist()
    tdf2[hai + ' derived Winsorized SIR'] = Winsorize_it(sirs, reported_winZ)
    tdf2[hai + ' derived W Z Score'] = ZScore_it(tdf2[hai + ' derived Winsorized SIR'], reported_winZ)
    
    # Assign maximum WinZ scores to hospitals with HAI footnote 18 
    maxWinZ = np.nanmax(tdf2[hai + ' derived W Z Score'])
    tdf3 = df_yr[df_yr[hai + ' Footnote'].isin([18, '18', '18 ', ' 18'])]
    tdf3[hai + ' derived Winsorized SIR'] = [np.nan]*tdf3.shape[0]
    tdf3[hai + ' derived W Z Score'] = [maxWinZ]*tdf3.shape[0]
    
    tdf4 = df_yr[df_yr[hai + ' Footnote'].isin([5, '5', ' 5', '5 ', 4, '4', ' 4', '4 '])]
    tdf4[hai + ' derived Winsorized SIR'] = [np.nan]*tdf4.shape[0]
    tdf4[hai + ' derived W Z Score'] = [np.nan]*tdf4.shape[0]
    
    df_yr = pd.concat([tdf2, tdf3, tdf4], axis=0)

del tdf2
del tdf3

display_df = df_yr.copy(deep=True)
items = ['file_year', 'HAI Measures End Date', 'HAI Measures Start Date',
         'CAUTI Footnote', 'CAUTI W Z Score', 'CAUTI derived W Z Score', 'CAUTI derived Winsorized SIR',
         'CDI Footnote', 'CDI W Z Score', 'CDI derived W Z Score',
         'CLABSI Footnote', 'CLABSI W Z Score', 'CLABSI derived W Z Score',
         'MRSA Footnote', 'MRSA W Z Score', 'MRSA derived W Z Score',
         #'Total HAC Footnote', 'Total HAC Score',
         ]


display_df = display_df.filter(items=items)
#display_df = display_df[display_df['CAUTI W Z Score'].isin([np.nan, float("NaN")])]
#display_df.sort_values(by='CAUTI derived W Z Score', inplace=True, ascending=False)
display_df.head(20)

,file_year,CAUTI Footnote,CAUTI W Z Score,CAUTI derived W Z Score,CAUTI derived Winsorized SIR,CDI Footnote,CDI W Z Score,CDI derived W Z Score,CLABSI Footnote,CLABSI W Z Score,CLABSI derived W Z Score,MRSA Footnote,MRSA W Z Score,MRSA derived W Z Score
0,2020,NaN,0.1160,0.114108,0.868600,NaN,-0.4571,-0.461837,NaN,1.0325,1.035504,NaN,-0.5533,-0.553208
2143,2020,NaN,1.1915,1.192865,1.435400,NaN,0.3543,0.354089,NaN,-0.1729,-0.172014,NaN,0.6308,0.633820
2144,2020,NaN,-0.4750,-0.476658,0.558200,NaN,-0.0424,-0.044211,NaN,-0.8572,-0.859545,NaN,0.8131,0.816440
2145,2020,NaN,-0.6099,-0.612740,0.486700,NaN,1.0184,1.021143,NaN,0.8097,0.813707,NaN,0.9299,0.932572
2147,2020,NaN,-0.5244,-0.526332,0.532100,NaN,-0.3519,-0.356449,NaN,-1.2092,-1.212386,NaN,-1.0807,-1.080678
2149,2020,NaN,-0.3990,-0.401480,0.597700,NaN,0.6728,0.672668,NaN,-0.3002,-0.300664,NaN,0.9865,0.990726
2150,2020,NaN,0.0628,0.061579,0.841000,NaN,-0.1476,-0.151109,NaN,-1.0441,-1.045240,NaN,-0.2931,-0.291690
2152,2020,NaN,-0.5453,-0.548029,0.520700,NaN,-0.6404,-0.644831,NaN,-1.1575,-1.160128,NaN,-0.4382,-0.438140
2153,2020,NaN,-0.5130,-0.515294,0.537900,NaN,0.9913,0.994871,NaN,-0.5290,-0.530440,NaN,0.0945,0.096599
2154,2020,NaN,-0.2071,-0.208301,0.699200,NaN,1.9710,1.978028,NaN,-0.8930,-0.894450,NaN,-0.1728,-0.170771


In [14]:

features = ['CAUTI derived SIR', 'CAUTI W Z Score', 'CAUTI derived W Z Score',
            'CDI derived SIR', 'CDI W Z Score', 'CDI derived W Z Score',
            'CLABSI derived SIR', 'CLABSI W Z Score', 'CLABSI derived W Z Score',
            'MRSA derived SIR', 'MRSA W Z Score', 'MRSA derived W Z Score', 
            'SSI W Z Score', 'PSI-90 W Z Score', 'Total HAC Score']

for f in features:
    df_yr[f] = df_yr[f].astype(str)
    df_yr[f] = pd.to_numeric(df_yr[f], errors='coerce')
    

for hai in hais:
    print(hai)
    
    ls1 = df_yr[hai + ' W Z Score'].tolist()
    ls2 = df_yr[hai + ' derived W Z Score'].tolist()
    final_ls = []
    
    for i, val in enumerate(ls1):
        if np.isnan(val) == False and np.isnan(ls2[i]) == True:
            final_ls.append(ls2[i])
        elif np.isnan(val) == False and np.isnan(ls2[i]) == False:
            final_ls.append(ls2[i])
        elif np.isnan(val) == True and np.isnan(ls2[i]) == False:
            final_ls.append(ls2[i])
        elif np.isnan(val) == True and np.isnan(ls2[i]) == True:
            final_ls.append(ls2[i])
        else:
            final_ls.append(ls2[i])
            
    df_yr[hai + ' derived W Z Score'] = final_ls


CAUTI
CLABSI
MRSA
CDI


## Attempt to reproduce HAC scores for 2020

In [15]:
print('Results from attempting to reproduce Yes/No penalty assignments:\n')
print('Excluded from results below:')
print('1. MD hospitals')
print('2. Hospitals with payment reduction values other than Yes or No\n')

df_yr.dropna(how='all', axis=1, inplace=True)

holdout_df = df_yr[(df_yr['State'] == 'MD') | ~df_yr['Payment Reduction'].isin(['Yes', 'No']) | (df_yr['Total HAC Score'].isin([float("NaN"), np.nan]))]
df_yr = df_yr[(df_yr['State'] != 'MD') & (df_yr['Payment Reduction'].isin(['Yes', 'No'])) & (~df_yr['Total HAC Score'].isin([float("NaN"), np.nan]))]

hac_scores = []
ct1 = 0
ct2 = 0


for hosp in df_yr['Facility ID'].tolist():
    tdf = df_yr[df_yr['Facility ID'] == hosp]

    d = 0
        
    w_ls = []
    sum_ls = []

    # Use original scores for all HAIs to test whether can penalties be reproduced when 
    # using data from the HACRP files
    #m_ls = ['CDI W Z Score', 'CAUTI W Z Score', 'CLABSI W Z Score', 'MRSA W Z Score', 'SSI W Z Score']

    # Use original SSI scores but derived scores for CDI, CAUTI, CLABSI, and MRSA for actual results
    m_ls = ['PSI-90 W Z Score', 'CDI derived W Z Score', 'CAUTI derived W Z Score', 'CLABSI derived W Z Score', 'MRSA derived W Z Score', 'SSI W Z Score']
                
    s = 0
    w = 0
    for m in m_ls:
        v = tdf[m].tolist()
        if len(list(set(v))) > 1:
            print('len(list(set(v))) > 1')
            sys.exit()

        v = tdf[m].iloc[0]

        if np.isnan(v) == False: 
            s += v
            w += 1

    if w == 0:
        d = np.nan
    else:
        d = s/w
        
    hac_scores.append(d)
    state = tdf['State'].iloc[0]
    
        
df_yr['Total HAC Score (derived)'] = hac_scores
print(df_yr.shape[0], 'hospitals in hac_df')


Results from attempting to reproduce Yes/No penalty assignments:

Excluded from results below:
1. MD hospitals
2. Hospitals with payment reduction values other than Yes or No

3149 hospitals in hac_df


In [16]:
p75 = np.nanpercentile(df_yr['Total HAC Score'], 75)
print('p75:', p75)

pr = []
for hosp in df_yr['Facility ID'].tolist():
    tdf = df_yr[df_yr['Facility ID'] == hosp]

    score = tdf['Total HAC Score'].iloc[0]

    if np.isnan(score) == True:
        pr.append('No')
    elif score <= p75:
        pr.append('No')
    elif score > p75:
        pr.append('Yes')
    else:
        print('This score is an error:', score)
        sys.exit()

df_yr['Payment Reduction (derived)'] = pr
    
o_list = df_yr['Payment Reduction'].tolist()
d_list = df_yr['Payment Reduction (derived)'].tolist()

same = 0
diff = 0
res_ls = []
for i, o in enumerate(o_list):
    if o == d_list[i]:
        same += 1
        res_ls.append(1)
    else:
        diff += 1
        res_ls.append(0)
            
df_yr['Payment Reduction Reproduced?'] = res_ls
    
print(same, "Penalty assignments were reproduced")
print(diff, "Penalty assignments were not reproduced")
print(str(np.round(100 * same/(same+diff), 2)) + '% penalty assignments were reproduced\n')
    
df_yr['HAC delta'] = df_yr['Total HAC Score'] - df_yr['Total HAC Score (derived)']
df_yr['CDI delta'] = df_yr['CDI W Z Score'] - df_yr['CDI derived W Z Score']
df_yr['MRSA delta'] = df_yr['MRSA W Z Score'] - df_yr['MRSA derived W Z Score']
df_yr['CAUTI delta'] = df_yr['CAUTI W Z Score'] - df_yr['CAUTI derived W Z Score']
df_yr['CLABSI delta'] = df_yr['CLABSI W Z Score'] - df_yr['CLABSI derived W Z Score']

df_yr.to_pickle('~/GitHub/HACRP-HAIs/data/merged_HAC_HAI/HAI_HAC_2020.pkl', protocol=5)


p75: 0.3305
3148 Penalty assignments were reproduced
1 Penalty assignments were not reproduced
99.97% penalty assignments were reproduced



In [17]:
ls1 = list(df_yr)
ls2 = list(holdout_df)
ls = list(filter(lambda x:x in ls1, ls2))
print(df_yr.shape)
print(holdout_df.shape)
df_yr = df_yr.merge(holdout_df, how='outer', on=ls)
print(df_yr.shape)


(3149, 59)
(69, 51)
(3218, 59)


In [18]:
tdf1 = df_yr[df_yr['Payment Reduction'] == 'Yes']
tdf2 = df_yr[df_yr['Payment Reduction'].isin(['Yes', 'No'])]
print(tdf1.shape[0]/tdf2.shape[0])

tdf1 = df_yr[df_yr['Payment Reduction'] == 'No']
print(tdf1.shape[0]/tdf2.shape[0])

print(df_yr.shape[0])
print(len(df_yr['Facility ID'].unique()))

df_yr.to_pickle('~/GitHub/HACRP-HAIs/data/merged_HAC_HAI/HAI_HAC_2020.pkl', protocol=5)


0.2478713339640492
0.7521286660359509
3218
3218
